Номер группы, ФИО:

Загрузить данные о перелетах. 

* Рассмотреть методы из лекции об ансамблевых моделях. Оценить разброс характеристики качества для каждого из методов с параметрами по умолчанию, проведя кросс-валидацию по 5ти блокам на тренировочный выборке.
* Выбрать один из методов для дальнейшей работы, демонстрирующий лучшее качество
* Подключить недостающие пакеты, выполнить при необходимости дополнительные преобразования
* Для выбранного метода подобрать лучшие значения гиперпараметров
* Оценить качество настроенного классификатора на тестовой выбоке

In [46]:
import numpy as np
import pandas as p
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [47]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [48]:
X = p.read_csv('train_n.csv', sep=',')
y = p.read_csv('train_n_y.csv', sep=',')
y = y['0']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

In [50]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [20]:
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [11]:
kfold = KFold(n_splits=5, random_state=5, shuffle=True)
model1 = RandomForestClassifier(n_estimators=100)
model2 = GradientBoostingClassifier(n_estimators=100)
results1 = cross_val_score(model1, X_train, y_train, cv=kfold) #0.96 
results2 = cross_val_score(model2, X_train, y_train, cv=kfold) #0.94

model1 = Perceptron(max_iter=1000, tol=1e-03)
model2 = LogisticRegression(solver='liblinear')
results1 = cross_val_score(model1, X_train_scaled, y_train, cv=kfold) #0.78 0.046
results2 = cross_val_score(model2, X_train_scaled, y_train, cv=kfold) #0.87 0.0023
model1 = SVC(gamma='auto') 
model2 = SVC(kernel='rbf', gamma='auto') 
results1 = cross_val_score(model1, X_train_scaled, y_train, cv=kfold)
results2 = cross_val_score(model2, X_train_scaled, y_train, cv=kfold)

KeyboardInterrupt: 

In [ ]:
print(results1, results1.mean(), results1.std())
print(results2, results2.mean(), results2.std())

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from plot_region import *

In [22]:
model1 = RandomForestClassifier()
model1.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [31]:
est=[100]
depth=[10,25,35]
feat=[5,10,15]

params={'n_estimators': est, 'max_depth': depth, 'max_features': feat}

grid_search = RandomizedSearchCV(model1, params, cv=5, 
                            return_train_score=True)
grid_search.fit(X_train_scaled, y_train)

/Users/arseniybileckiy/PycharmProjects/Database3/venv/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [10, 25, 35],
                                        'max_features': [5, 10, 15],
                                        'n_estimators': [100]},
                   return_train_score=True)

In [39]:
GetDfFromCvResults(grid_search).sort_values(by='mean_test_score', ascending=False)

/Users/arseniybileckiy/PycharmProjects/Database3/ML_4c/HW_5/plot_region.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  .join(p.io.json.json_normalize(grid_search.cv_results_["params"]).add_prefix("param_")))


,mean_test_score,mean_fit_time,param_n_estimators,param_max_features,param_max_depth
4,0.962740,11.385206,100,10,25
8,0.962382,16.063210,100,15,35
7,0.962272,11.417795,100,10,35
5,0.961860,15.912586,100,15,25
6,0.961159,6.948004,100,5,35
3,0.961159,6.882530,100,5,25
2,0.952689,12.044968,100,15,10
1,0.951768,8.766730,100,10,10
0,0.946475,5.329484,100,5,10


In [51]:
y_pred =grid_search.best_estimator_.predict(X_test_scaled)

In [52]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.964808161170281